In [1]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


In [2]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value"
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


In [3]:
model_name = 'openlm-research/open_llama_7b_v2'
model, peft_config, tokenizer = create_and_prepare_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# model = PeftModel.from_pretrained(model, './results/teste_1000iteracoes/checkpoint-1000')

In [4]:
gen_input_1 = lambda x: f"### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. \
                Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. \
                Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa, \
                empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e \
                que ele poderá voltar a fazer pedidos no restaurante. \
                Certifique-se de usar detalhes específicos do comentário do cliente.\n \
                <{x}>\n\
                ### Reply: "

gen_input_2 = lambda x: f"Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}"

gen_input_3 = lambda x: f"Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: {x}"

In [5]:
def inference(model, text):
    text_token = tokenizer(
    text,
    padding=True,
    truncation=True,
    return_tensors='pt'
    )
    text_token = text_token.to('cuda:0')

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            input_ids = text_token.input_ids, 
            max_new_tokens=200,
            # temperature=0.1,
            top_p=0.9,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    print(generated_text)

## prompt 1

In [7]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_1(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Alimentação caríssima para um péssimo atendimento.. super mal atendido>
                ### Reply: "Obrigado por nos dar a oportunidade de melhorar nosso serviço. \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n


In [8]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!>
                ### Reply: <Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!>
                ### Reply: <Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de pe

In [9]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.>
                ### Reply: "Obrigado por nos ter chamado para resolver seu problema. \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\


In [10]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_1(comment))

### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante.                 Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante.                 Dado o comentário do cliente entre, escreva em Português um comentário de resposta de forma respeitosa,                 empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e                 que ele poderá voltar a fazer pedidos no restaurante.                 Certifique-se de usar detalhes específicos do comentário do cliente.
                 <Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.>
                ### Reply: "Obrigado por nos ter chamado. Nós vamos tentar resolver seu problema o mais rápido possível. Nós vamos tentar resolver seu problema o mais rápido possível. N

## prompt 2

In [6]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.
Agradecemos a sua opinião.



In [7]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!










































































































































































































In [8]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.

A resposta deve ser escrita em forma de e-mail e deve ser enviada para o e-mail do cliente.

A resposta deve ser escrita em forma de e-mail e deve ser enviada para o e-mail do cliente.

A resposta deve ser escrita em forma de e-mail e deve ser enviada para o e-mail do cliente.

A resposta deve ser escrita em forma de e-mail e deve ser enviada para o e-mail do cliente.

A resposta deve ser escrita em forma de e-mail e deve ser enviada para o e-mail do cliente.

A resposta deve ser escrita em forma de e-mail e deve ser enviada para o e-mail


In [14]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_2(comment))

Escreva em Português um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
O restaurante é um restaurante de hambúrguer, com 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espetáculos.
O restaurante tem 2 salas de jantar e 1 salão de espet


## Prompt 3

In [15]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Alimentação caríssima para um péssimo atendimento.. super mal atendido.
O cliente escreveu:
I was in your restaurant last night with my family. We were very disappointed with the service. We were seated at a table that was not cleaned. We had to ask for a clean table. We were not offered a menu. We had to ask for a menu. We were not offered a drink menu. We had to ask for a drink menu. We were not offered a bread basket. We had to ask for a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. We were not offered a bread basket. W

In [16]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.
O cliente não gostou do frango.


In [17]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
O cliente escreveu:
“I was there last night with my wife and we had a great time. The food was excellent and the service was great. The only problem was that the place was very hot and the food took a long time to get to the table. I would recommend this place to anyone who wants to have a great time and eat some great food.”
O cliente escreveu:
“I was there last night with my wife and we had a great time. The food was excellent and the service was great. The only problem was that the place was very hot and the food took a long time to get to the table. I would recommend this place to anyone who wants to have a great time and eat some great food.”
O cliente escreveu:
“I was there last night with my wife and we had a great time. The food was excellent and the service was great. The only pro

In [18]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input_3(comment))

Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
O restaurante é um restaurante de hambúrguer, com 2 salas de jantar, uma para os clientes internos e outra para os clientes externos.
O restaurante tem 2 funcionários de atendimento, um para cada sala.
O funcionário interno tem 2 salas de jantar para atender, uma com 10 mesas e a outra com 8 mesas.
O funcionário externo tem 1 sala de jantar para atender, com 8 mesas.
O funcionário interno tem 1 funcionário externo para ajudar, com 1 funcionário interno para ajudar.
O funcionário externo tem 1 funcionário interno para ajudar, com 1 funcionário interno para ajudar.
O funcionário interno tem 1 funcionário externo para ajudar, com 1 funcionário interno para ajudar
